In [1]:
# Create a plot as needed 
### YOUR CODE HERE ###
def CorrelationHeatMap(df0) :
    # Plot a correlation heatmap
    plt.figure(figsize=(10, 6))
    heatmap = sns.heatmap(df0.corr(), vmin=-1, vmax=1, annot=True, cmap=sns.color_palette("vlag", as_cmap=True))
    heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':14}, pad=12);

In [2]:
def renamefunction() : 
  print('Rename Function Called')
  '''
    df0 = df0.rename(columns={'Work_accident': 'work_accident',
                          'average_montly_hours': 'average_monthly_hours',
                          'time_spend_company': 'tenure',
                          'Department': 'department'})
   ''' 
  return()

In [3]:
def make_results(model_name:str, model_object, metric:str):
    '''
    Arguments:
    model_name (string): what you want the model to be called in the output table
    model_object: a fit GridSearchCV object
    metric (string): precision, recall, f1, or accuracy

    Returns a pandas df with the F1, recall, precision, and accuracy scores
    for the model with the best mean 'metric' score across all validation folds.
    '''

    # Create dictionary that maps input metric to actual metric name in GridSearchCV
    metric_dict = {'precision': 'mean_test_precision',
                 'recall': 'mean_test_recall',
                 'f1': 'mean_test_f1',
                 'accuracy': 'mean_test_accuracy',
                 }

    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)

    # Isolate the row of the df with the max(metric) score
    best_estimator_results = cv_results.iloc[cv_results[metric_dict[metric]].idxmax(), :]

    # Extract Accuracy, precision, recall, and f1 score from that row
    f1 = best_estimator_results.mean_test_f1
    recall = best_estimator_results.mean_test_recall
    precision = best_estimator_results.mean_test_precision
    accuracy = best_estimator_results.mean_test_accuracy

    # Create table of results
    table = pd.DataFrame({'model': [model_name],
                        'precision': [precision],
                        'recall': [recall],
                        'F1': [f1],
                        'accuracy': [accuracy],
                        },
                       )

    return table

In [4]:

def plot_model_Tree_save_image (rf_cv) :
    from sklearn import tree
    fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=800)
    tree.plot_tree(rf_cv.best_estimator_.estimators_[0])
    fig.savefig('rf_individualtree.png')

def plot_model_Tree(rf_cv,X):
    from sklearn import tree
    # Plot the tree
    plt.figure(figsize=(85,20))
    tree.plot_tree(rf_cv.best_estimator_, max_depth=6, fontsize=14, feature_names=X.columns, 
          class_names={0:'stayed', 1:'left'}, filled=True);
    plt.show()

In [4]:
def PlotTreeImportance(clf,X):
    #tree2_importances = pd.DataFrame(clf.best_estimator_.feature_importances_, columns=X.columns)
    tree2_importances = pd.DataFrame(clf.best_estimator_.feature_importances_, 
                                    columns=['gini_importance'], 
                                    index=X.columns
                                    )
    tree2_importances = tree2_importances.sort_values(by='gini_importance', ascending=False)

    # Only extract the features with importances > 0
    tree2_importances = tree2_importances[tree2_importances['gini_importance'] != 0]
    tree2_importances



def GetFeatureImportance(clf_list,Model_Names_list,X,top_n_features):
    import numpy as np
    import pandas as pd
    fig, axes = plt.subplots(1, 3, figsize=(25, 4))  # Adjust figsize as needed

    for i in range(0,len(clf_list),1):
        clf = clf_list[i]
        Model_Name = Model_Names_list[i]
        # Get feature importances
        feat_impt = clf.best_estimator_.feature_importances_

        # Get indices of top 10 features
        ind = np.argpartition(clf.best_estimator_.feature_importances_, -top_n_features)[-top_n_features:]

        # Get column labels of top 10 features 
        feat = X.columns[ind]

        # Filter `feat_impt` to consist of top 10 feature importances
        feat_impt = feat_impt[ind]

        y_df = pd.DataFrame({"Feature":feat,"Importance":feat_impt})
        y_sort_df = y_df.sort_values("Importance")
        #fig = plt.figure()
        #axes[i] = fig.add_subplot(111)

        y_sort_df.plot(ax=axes[i],kind='barh',x="Feature",y="Importance")

        axes[i].set_title(f"Feature Importances {Model_Name}" , fontsize=10)
        axes[i].set_ylabel("Feature")
        axes[i].set_xlabel("Importance")

    plt.show()

In [13]:
def get_scores(model_name:str, model, X_test_data, y_test_data,best_estimatorUse):
    
    #Generate a table of test scores.

    #In: 
       # model_name (string):  How you want your model to be named in the output table
       # model:                A fit GridSearchCV object
       # X_test_data:          numpy array of X_test data
       # y_test_data:          numpy array of y_test data

    #Out: pandas df of precision, recall, f1, accuracy, and AUC scores for your model
    



    if (best_estimatorUse == True) :
        preds = model.best_estimator_.predict(X_test_data)
    else :
        preds = model.predict(X_test_data)


    auc = roc_auc_score(y_test_data, preds)
    accuracy = accuracy_score(y_test_data, preds)
    precision = precision_score(y_test_data, preds)
    recall = recall_score(y_test_data, preds)
    f1 = f1_score(y_test_data, preds)

    table = pd.DataFrame({'model': [model_name],
                          'precision': [precision], 
                          'recall': [recall],
                          'f1': [f1],
                          'accuracy': [accuracy],
                          'AUC': [auc]
                         })

    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
    cm=confusion_matrix(y_test_data,preds)
    #plot_confusion_matrix(cm,model_name)

    
    #disp=ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model.classes_)
    #disp.plot(values_format='') # `values_format=''` suppresses scientific notation
    #plt.show()
    '''
    title = model_name

    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax)
    ax.set_title(title)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    '''  
    return table,cm

In [ ]:
# Define a path to the folder where you want to save the model
import pickle
def write_pickle(path, model_object, save_name:str):
    '''
    save_name is a string.
    '''
    with open(path + save_name + '.pickle', 'wb') as to_write:
        pickle.dump(model_object, to_write)
        

def read_pickle(path, saved_model_name:str):
    '''
    saved_model_name is a string.
    '''
    with open(path + saved_model_name + '.pickle', 'rb') as to_read:
        model = pickle.load(to_read)

        return model
    

In [ ]:

def Impute_upper_limit(column_list,iqr_factor,Q1_Value,Q3_value) :
    iqr_value = Q3_value-Q1_Value
    print("iqr_value = ",iqr_value)
    upper_limit = iqr_factor*iqr_value + Q3_value
    print("upper_limit = ",upper_limit)
    for i in range(len(column_list) ):
        #print(column_list)
        if (column_list[i] > upper_limit) :
            column_list[i] = upper_limit     
         
   # column_list = column_list.mask(column_list>upper_limit,upper_limit)
   # print("column_list = ",column_list)
    return(column_list)

def Impute_lower_limit(column_list,iqr_factor,Q1_Value,Q3_value) :
    iqr_value = Q3_value-Q1_Value
    print("iqr_value = ",iqr_value)
    lower_limit = Q1_Value-iqr_factor*iqr_value 
    print("lower_limit = ",lower_limit)
    for i in range(len(column_list) ):
        #print(column_list)
        if (column_list[i] < lower_limit) :
            column_list[i] = lower_limit     
         
   # column_list = column_list.mask(column_list>upper_limit,upper_limit)
   # print("column_list = ",column_list)
    return(column_list)

def Impute_lower_limit_to_threshold(column_list,threshold_value) :

    for i in range(len(column_list) ):
        #print(column_list)
        if (column_list[i] < threshold_value) :
            column_list[i] = threshold_value     
         
   # column_list = column_list.mask(column_list>upper_limit,upper_limit)
   # print("column_list = ",column_list)
    return(column_list)


def Impute_missing_values(column_list,impute_value) :
    import math
    print("impute missiong values by = ",impute_value)

    for i in range(len(column_list) ):
        #print(column_list)
        if (np.isnan(column_list[i])==True) :
            column_list[i] = impute_value     
         
   # column_list = column_list.mask(column_list>upper_limit,upper_limit)
   # print("column_list = ",column_list)
    return(column_list)